<a href="https://colab.research.google.com/github/PrinceChaudhary1962/CPU-Scheduling/blob/main/CPU_Scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
%%writefile cpu_scheduling.cpp
#include <iostream>
#include <vector>
#include <algorithm>
#include <queue>
#include <iomanip>
using namespace std;

struct Process {
    int pid;
    int arrival;
    int burst;
    int priority;
    int start;
    int finish;
    int waiting;
    int turnaround;
    int remaining;
};

bool sortByArrival(Process a, Process b) { return a.arrival < b.arrival; }
bool sortByBurst(Process a, Process b) { return a.burst < b.burst; }
bool sortByPriority(Process a, Process b) { return a.priority < b.priority; }

void printTable(const vector<Process>& p) {
    cout << "\nPID\tAT\tBT\tPR\tWT\tTAT\n";
    for (const auto& pr : p) {
        cout << pr.pid << "\t" << pr.arrival << "\t" << pr.burst << "\t" << pr.priority
             << "\t" << pr.waiting << "\t" << pr.turnaround << "\n";
    }
}

void FCFS(vector<Process> p) {
    sort(p.begin(), p.end(), sortByArrival);
    int current = 0;
    for (auto& pr : p) {
        pr.start = max(current, pr.arrival);
        pr.finish = pr.start + pr.burst;
        pr.waiting = pr.start - pr.arrival;
        pr.turnaround = pr.finish - pr.arrival;
        current = pr.finish;
    }
    cout << "\n--- FCFS Scheduling ---";
    printTable(p);
}

void SJFNonPreemptive(vector<Process> p) {
    sort(p.begin(), p.end(), sortByArrival);
    int current = 0;
    vector<Process> result;
    while (!p.empty()) {
        auto it = min_element(p.begin(), p.end(), [&](Process a, Process b) {
            if (a.arrival > current && b.arrival > current) return a.arrival < b.arrival;
            if (a.arrival <= current && b.arrival > current) return true;
            if (b.arrival <= current && a.arrival > current) return false;
            return a.burst < b.burst;
        });
        if (it->arrival > current) current = it->arrival;
        it->start = current;
        it->finish = current + it->burst;
        it->waiting = it->start - it->arrival;
        it->turnaround = it->finish - it->arrival;
        current = it->finish;
        result.push_back(*it);
        p.erase(it);
    }
    cout << "\n--- SJF (Non-Preemptive) Scheduling ---";
    printTable(result);
}

void SJFPreemptive(vector<Process> p) {
    int time = 0, completed = 0, n = p.size();
    for (auto& pr : p) pr.remaining = pr.burst;

    while (completed < n) {
        int idx = -1, minBurst = 1e9;
        for (int i = 0; i < n; ++i) {
            if (p[i].arrival <= time && p[i].remaining > 0 && p[i].remaining < minBurst) {
                minBurst = p[i].remaining;
                idx = i;
            }
        }
        if (idx == -1) {
            ++time;
            continue;
        }
        if (p[idx].remaining == p[idx].burst) p[idx].start = time;
        --p[idx].remaining;
        ++time;
        if (p[idx].remaining == 0) {
            ++completed;
            p[idx].finish = time;
            p[idx].turnaround = p[idx].finish - p[idx].arrival;
            p[idx].waiting = p[idx].turnaround - p[idx].burst;
        }
    }
    cout << "\n--- SJF (Preemptive) Scheduling ---";
    printTable(p);
}

void PriorityScheduling(vector<Process> p) {
    sort(p.begin(), p.end(), sortByArrival);
    int current = 0;
    vector<Process> result;
    while (!p.empty()) {
        auto it = min_element(p.begin(), p.end(), [&](Process a, Process b) {
            if (a.arrival > current && b.arrival > current) return a.arrival < b.arrival;
            if (a.arrival <= current && b.arrival > current) return true;
            if (b.arrival <= current && a.arrival > current) return false;
            return a.priority < b.priority;
        });
        if (it->arrival > current) current = it->arrival;
        it->start = current;
        it->finish = current + it->burst;
        it->waiting = it->start - it->arrival;
        it->turnaround = it->finish - it->arrival;
        current = it->finish;
        result.push_back(*it);
        p.erase(it);
    }
    cout << "\n--- Priority Scheduling ---";
    printTable(result);
}

void RoundRobin(vector<Process> p, int quantum) {
    int time = 0, completed = 0, n = p.size();
    queue<int> q;
    vector<bool> inQueue(n, false);
    vector<Process> result = p;

    for (auto& pr : result) pr.remaining = pr.burst;

    q.push(0);
    inQueue[0] = true;
    time = result[0].arrival;

    while (!q.empty()) {
        int idx = q.front(); q.pop();

        if (result[idx].remaining > quantum) {
            result[idx].remaining -= quantum;
            time += quantum;
        } else {
            time += result[idx].remaining;
            result[idx].remaining = 0;
            result[idx].finish = time;
            result[idx].turnaround = result[idx].finish - result[idx].arrival;
            result[idx].waiting = result[idx].turnaround - result[idx].burst;
            completed++;
        }

        for (int i = 0; i < n; ++i) {
            if (!inQueue[i] && result[i].arrival <= time && result[i].remaining > 0) {
                q.push(i);
                inQueue[i] = true;
            }
        }

        if (result[idx].remaining > 0) q.push(idx);
    }

    cout << "\n--- Round Robin Scheduling (Time Quantum = " << quantum << ") ---";
    printTable(result);
}

int main() {
    int n;
    cout << "Enter number of processes: ";
    cin >> n;
    vector<Process> processes(n);
    for (int i = 0; i < n; ++i) {
        processes[i].pid = i + 1;
        cout << "Enter arrival time, burst time, and priority for process " << i + 1 << ": ";
        cin >> processes[i].arrival >> processes[i].burst >> processes[i].priority;
    }

    int tq1 = 2, tq2 = 4;
    FCFS(processes);
    SJFNonPreemptive(processes);
    SJFPreemptive(processes);
    PriorityScheduling(processes);
    RoundRobin(processes, tq1);
    RoundRobin(processes, tq2);

    return 0;
}


Overwriting cpu_scheduling.cpp


In [29]:
!g++ cpu_scheduling.cpp -o cpu_scheduling


In [30]:
!./cpu_scheduling


Enter number of processes: 3
Enter arrival time, burst time, and priority for process 1: 1
2
3
Enter arrival time, burst time, and priority for process 2: 2
1
2
Enter arrival time, burst time, and priority for process 3: 4
2
1

--- FCFS Scheduling ---
PID	AT	BT	PR	WT	TAT
1	1	2	3	0	2
2	2	1	2	1	2
3	4	2	1	0	2

--- SJF (Non-Preemptive) Scheduling ---
PID	AT	BT	PR	WT	TAT
1	1	2	3	0	2
2	2	1	2	1	2
3	4	2	1	0	2

--- SJF (Preemptive) Scheduling ---
PID	AT	BT	PR	WT	TAT
1	1	2	3	0	2
2	2	1	2	1	2
3	4	2	1	0	2

--- Priority Scheduling ---
PID	AT	BT	PR	WT	TAT
1	1	2	3	0	2
2	2	1	2	1	2
3	4	2	1	0	2

--- Round Robin Scheduling (Time Quantum = 2) ---
PID	AT	BT	PR	WT	TAT
1	1	2	3	0	2
2	2	1	2	1	2
3	4	2	1	0	2

--- Round Robin Scheduling (Time Quantum = 4) ---
PID	AT	BT	PR	WT	TAT
1	1	2	3	0	2
2	2	1	2	1	2
3	4	2	1	0	2
